## Environment Setup

In [4]:
!brew install openai
!brew install datasets
!brew install evaluate

==> Downloading https://formulae.brew.sh/api/formula.jws.json
######################################################################### 100.0%
==> Downloading https://formulae.brew.sh/api/cask.jws.json
######################################################################### 100.0%
==> Searching for similarly named formulae and casks...
==> Formulae
openai-whisper      openvi              opencc              opensc
openfa              openbao             opencv              openj9

To install openai-whisper, run:
  brew install openai-whisper

==> Casks
openbci   opencat   openlp    openmw    openra    opensc    opensim   opera

To install openbci, run:
  brew install --cask openbci
==> Searching for similarly named formulae and casks...
==> Formulae
datasette

To install datasette, run:
  brew install datasette

==> Casks
datasette

To install datasette, run:
  brew install --cask datasette
==> Searching for similarly named formulae and casks...
Error: No formulae or casks found for e

In [7]:
import json
from openai import OpenAI
from IPython.utils.capture import capture_output
from datasets import load_dataset
import pprint

/Users/aditichandrashekar/Library/CloudStorage/OneDrive-CaliforniaInstituteofTechnology/Term9/LegalSumAI/prompting_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from google.colab import drive
drive.mount('/content/drive')


%cd /content/drive/MyDrive/CS 159 Project

ModuleNotFoundError: No module named 'google'

In [5]:
from factsheet_utils import *
from metrics_utils import *
from summary_utils import *

User token is required.

In [17]:
client = OpenAI(api_key = input())

## Input Source Text

In [23]:
raw_case = input('Paste raw case file here: ')

In [24]:
pprint.pprint(raw_case)

('Kendall Jenner and her NBA player boyfriend of two years, Devin Booker, are '
 'done dating for now, E!, Entertainment Tonight, and Us Weekly report. '
 'Sources spoke to all three outlets about what caused the sudden split, which '
 'happened less than two weeks ago. Collectively, they made it clear that '
 'Jenner and Booker could reconcile but clashed over what they wanted for '
 'their future. Jenner initiated the split, E! reported.  devin booker and '
 'kendall jenner in portofino NINO//GETTY IMAGES Devin Booker and Kendall '
 'Jenner in Portofino for Kourtney Kardashian’s wedding last month. A source '
 "told Entertainment Tonight, “Kendall feels like they're on different paths.” "
 'A second source added that “[they] have had discussions about their future, '
 'but they are not on the same page.”  A source echoed that to Us Weekly, '
 'saying that Jenner and Booker want to “figure out if a future together is '
 "what’s meant to be. [They're] tak[ing] this time to focus on the

## Generate Factsheets

In [25]:
# Break source up into segments for processing
print('Generating fact sheet for each document in source...')
source = break_text(raw_case)
source_responses = []
for s in source:
  source_responses.append(generate_fact_sheet(s, client, verbose=False))

# combine source responses
print('Combining source fact sheets...')
source_sheet = combine_fact_sheets(source_responses, client, verbose=False)
print(source_sheet)

Generating fact sheet for each document in source...
Combining source fact sheets...
```json
{
    "Case Information": "Kendall Jenner and her NBA player boyfriend of two years, Devin Booker, have recently split after facing differences in their future plans. Jenner initiated the split, citing that they were on different paths and needed to focus on themselves.",
    "Parties Involved": "Kendall Jenner, Devin Booker",
    "Issue": "The main legal question in the case is not specified as there seems to be no legal dispute and it appears to be a personal relationship matter.",
    "Rule": "No specific legal rules or principles mentioned as the case pertains to a personal relationship matter with no legal dispute outlined.",
    "Application": "Given the absence of a legal dispute and the personal nature of the case, there are no legal rules or principles to apply.",
    "Conclusion": "The outcome of the split between Kendall Jenner and Devin Booker is that they are currently not together

## Generate Summary

We can generate summaries now with Chain of Density Prompting. The user can adjust parameters such as approximate length of summary output (using the `num_words` parameter) and desired audience (using the `initial_prompt` parameter).

In [26]:
num_words = input('Number of words (Tiny - 25, Short - 130, Long - 650): ')
inital_prompt = "Summarize the factsheet so that it is understandable to the general public. Refer to the case by official name (partyX v. partyY)."
responses = chain_of_density_prompting(source_sheet, inital_prompt, client, ground_truth=None, num_words=int(num_words))
pprint.pprint(responses[-1][0])

('In the court case involving Kendall Jenner and Devin Booker, the split arose '
 'due to conflicting future aspirations, prompting Jenner to end the '
 'relationship, emphasizing individual growth. The legal aspect remains '
 'absent, with no specific legal dispute outlined, framing the issue as '
 'personal. Jenner and Booker are presently separated, utilizing this time for '
 "introspection, implying a possible reunion. Despite Booker's interactions "
 'with other women post-breakup, Jenner maintains a cordial relationship while '
 "seeking space. The court's decision reflects the personal nature of the "
 'case, emphasizing individual paths over legal complexities, leading to their '
 'current separation. Both parties are exploring the viability of their bond, '
 'showing potential for reconciliation amidst their personal evaluations.')
